# Trabajo Practico de Econometria
**Maestrias en Economia y Econometria**

Seed: 1910

In [1]:
import numpy as np
from scipy import stats

# Seed para replicabilidad
SEED = 1910

# Parametros del modelo
N_SIM = 5000
BETA0_TRUE = -3.0
BETA1_TRUE = 0.8
OMEGA_DIAG = np.array([4.0, 9.0, 16.0, 25.0, 36.0])  # varianzas por grupo
N_GROUPS = 5

## Parte 1: Propiedades de muestra finita de FGLS (MCGE)

**Modelo:** $y_i = \beta_0 + \beta_1 x_i + u_i$, con $\beta_0 = -3$, $\beta_1 = 0.8$

$u \sim N(0, \Omega \otimes I_N)$, $\Omega = \text{diag}(4, 9, 16, 25, 36)$, $x \sim U[1, 50]$

In [2]:
def generate_data(n_total, beta0, beta1, rng):
    """
    Genera una muestra de tamano n_total = 5*N del modelo (1).
    Cada grupo j (j=0,...,4) tiene N = n_total/5 observaciones con varianza Omega[j,j].
    """
    N = n_total // N_GROUPS
    x = rng.uniform(1, 50, size=n_total)

    # Errores con heterocedasticidad grupal
    u = np.zeros(n_total)
    for j in range(N_GROUPS):
        sigma_j = np.sqrt(OMEGA_DIAG[j])
        u[j * N:(j + 1) * N] = rng.normal(0, sigma_j, size=N)

    y = beta0 + beta1 * x + u
    return y, x


def ols(y, X):
    """Estimacion por MCC (OLS)."""
    return np.linalg.solve(X.T @ X, X.T @ y)


def fgls(y, X, n_total):
    """
    Estimacion por FGLS (MCGE).
    1. Estimar OLS y obtener residuos
    2. Estimar varianza por grupo con residuos al cuadrado (correccion por g.d.l.)
    3. GLS con Omega_hat estimada
    Retorna coeficientes y errores estandar.
    """
    N = n_total // N_GROUPS
    k = X.shape[1]  # numero de regresores (2: constante + x)

    # Paso 1: OLS
    beta_ols = ols(y, X)
    residuals = y - X @ beta_ols

    # Paso 2: Estimar varianzas por grupo con correccion por grados de libertad
    # Multiplicamos por n/(n-k) para corregir el sesgo de los residuos OLS
    df_correction = n_total / (n_total - k)
    sigma2_hat = np.zeros(n_total)
    for j in range(N_GROUPS):
        idx = slice(j * N, (j + 1) * N)
        sigma2_j = np.mean(residuals[idx] ** 2) * df_correction
        sigma2_hat[idx] = sigma2_j

    # Paso 3: GLS -> beta_hat = (X'Omega_inv X)^{-1} X'Omega_inv y
    Omega_inv = np.diag(1.0 / sigma2_hat)
    XtOiX = X.T @ Omega_inv @ X
    XtOiX_inv = np.linalg.inv(XtOiX)
    beta_fgls = XtOiX_inv @ (X.T @ Omega_inv @ y)

    # Varianza FGLS: (X'Omega_inv X)^{-1}
    se_beta = np.sqrt(np.diag(XtOiX_inv))

    return beta_fgls, se_beta

In [ ]:
def run_simulation_1a(n_total):
    """
    Simulacion punto 1a para un tamano de muestra dado.
    FGLS grupos: conoce la estructura de grupos, estima varianzas por grupo.
    - Tamano del test: H0: beta1 = 0.8 (datos generados con beta1 = 0.8)
    - Poder del test: datos generados con beta1 = 0 y beta1 = 0.4
    Usa valores criticos de la normal estandar (distribucion asintotica).
    """
    beta0_fgls   = np.zeros(N_SIM);  beta1_fgls   = np.zeros(N_SIM)
    t_size_fgls  = np.zeros(N_SIM)
    t_pow0_fgls  = np.zeros(N_SIM);  t_pow04_fgls = np.zeros(N_SIM)

    rng = np.random.RandomState(SEED)

    for sim in range(N_SIM):
        # --- Bajo H0: beta1 = 0.8 ---
        y, x = generate_data(n_total, BETA0_TRUE, BETA1_TRUE, rng)
        X = np.column_stack([np.ones(n_total), x])
        b_f, se_f = fgls(y, X, n_total)
        beta0_fgls[sim] = b_f[0];  beta1_fgls[sim] = b_f[1]
        t_size_fgls[sim] = (b_f[1] - BETA1_TRUE) / se_f[1]

        # --- Poder: beta1 = 0 ---
        y0, x0 = generate_data(n_total, BETA0_TRUE, 0.0, rng)
        X0 = np.column_stack([np.ones(n_total), x0])
        b0_f, se0_f = fgls(y0, X0, n_total)
        t_pow0_fgls[sim] = (b0_f[1] - BETA1_TRUE) / se0_f[1]

        # --- Poder: beta1 = 0.4 ---
        y04, x04 = generate_data(n_total, BETA0_TRUE, 0.4, rng)
        X04 = np.column_stack([np.ones(n_total), x04])
        b04_f, se04_f = fgls(y04, X04, n_total)
        t_pow04_fgls[sim] = (b04_f[1] - BETA1_TRUE) / se04_f[1]

    cv_1 = stats.norm.ppf(1 - 0.01 / 2)
    cv_5 = stats.norm.ppf(1 - 0.05 / 2)

    return {
        'n_total': n_total,
        'b0_mean': np.mean(beta0_fgls),   'b0_med': np.median(beta0_fgls),   'b0_std': np.std(beta0_fgls),
        'b1_mean': np.mean(beta1_fgls),   'b1_med': np.median(beta1_fgls),   'b1_std': np.std(beta1_fgls),
        'size_1':  np.mean(np.abs(t_size_fgls)  > cv_1) * 100,
        'size_5':  np.mean(np.abs(t_size_fgls)  > cv_5) * 100,
        'pow0_1':  np.mean(np.abs(t_pow0_fgls)  > cv_1) * 100,
        'pow0_5':  np.mean(np.abs(t_pow0_fgls)  > cv_5) * 100,
        'pow04_1': np.mean(np.abs(t_pow04_fgls) > cv_1) * 100,
        'pow04_5': np.mean(np.abs(t_pow04_fgls) > cv_5) * 100,
    }


def print_results(res):
    """Imprime resultados FGLS grupos."""
    W = 55
    print("=" * W)
    print(f"  FGLS grupos  |  5N = {res['n_total']}")
    print("=" * W)

    print(f"\n--- beta0 (verdadero = {BETA0_TRUE}) ---")
    print(f"  Media:   {res['b0_mean']:12.6f}")
    print(f"  Mediana: {res['b0_med']:12.6f}")
    print(f"  Desvio:  {res['b0_std']:12.6f}")

    print(f"\n--- beta1 (verdadero = {BETA1_TRUE}) ---")
    print(f"  Media:   {res['b1_mean']:12.6f}")
    print(f"  Mediana: {res['b1_med']:12.6f}")
    print(f"  Desvio:  {res['b1_std']:12.6f}")

    print(f"\n--- Tamano del test (H0: beta1 = {BETA1_TRUE}) ---")
    print(f"  Al 1%  (nominal: 1%):  {res['size_1']:6.2f}%")
    print(f"  Al 5%  (nominal: 5%):  {res['size_5']:6.2f}%")

    print(f"\n--- Poder del test (H0: beta1 = {BETA1_TRUE}) ---")
    print(f"  beta1=0,   al 1%:  {res['pow0_1']:6.2f}%")
    print(f"  beta1=0,   al 5%:  {res['pow0_5']:6.2f}%")
    print(f"  beta1=0.4, al 1%:  {res['pow04_1']:6.2f}%")
    print(f"  beta1=0.4, al 5%:  {res['pow04_5']:6.2f}%")


In [6]:
for n in [5,10,30,100,200,500]:
    res_5 = run_simulation_1a(n_total=n)
    print_results(res_5)

  FGLS - Tamano de muestra: 5N = 5

--- Estimaciones de beta0 (verdadero = -3.0) ---
  Media:    -2.917793
  Mediana:  -2.934760
  Desvio:   5.597167

--- Estimaciones de beta1 (verdadero = 0.8) ---
  Media:    0.794782
  Mediana:  0.795753
  Desvio:   0.192855

--- Tamano del test (H0: beta1 = 0.8) ---
  Al 1%:  28.64%  (nominal: 1%)
  Al 5%:  38.30%  (nominal: 5%)

--- Poder del test (H0: beta1 = 0.8) ---
  beta1 = 0:    al 1%: 96.96%  |  al 5%: 98.34%
  beta1 = 0.4:  al 1%: 80.82%  |  al 5%: 87.68%
  FGLS - Tamano de muestra: 5N = 10

--- Estimaciones de beta0 (verdadero = -3.0) ---
  Media:    -2.966155
  Mediana:  -3.001572
  Desvio:   3.034392

--- Estimaciones de beta1 (verdadero = 0.8) ---
  Media:    0.799587
  Mediana:  0.799594
  Desvio:   0.105066

--- Tamano del test (H0: beta1 = 0.8) ---
  Al 1%:  16.28%  (nominal: 1%)
  Al 5%:  25.90%  (nominal: 5%)

--- Poder del test (H0: beta1 = 0.8) ---
  beta1 = 0:    al 1%: 99.96%  |  al 5%: 100.00%
  beta1 = 0.4:  al 1%: 95.66%  |

### Punto 1b: Descomposicion de Cholesky

Se busca $P$ tal que $\Omega = P \cdot P'$. Luego se transforma el modelo:

$$P^{-1} y = P^{-1} X \beta + P^{-1} u$$

donde $P^{-1} u$ tiene covarianza $P^{-1} \Omega (P^{-1})' = P^{-1} P P' (P^{-1})' = I$.

Al estimar por OLS el modelo transformado ($y^* = X^* \beta + u^*$) se obtiene el estimador GLS.

In [ ]:
def run_simulation_1b(n_total=5):
    """
    Punto 1b: GLS via Cholesky + OLS sobre modelo transformado.
    Usa Omega verdadera (conocida), no estimada.
    Compara con FGLS grupos del punto 1a.
    """
    N = n_total // N_GROUPS

    # Construir Omega completa (5N x 5N): diagonal con varianzas por grupo
    omega_full = np.diag(np.repeat(OMEGA_DIAG, N))

    # Descomposicion de Cholesky: Omega = P * P'
    P = np.linalg.cholesky(omega_full)
    P_inv = np.linalg.inv(P)

    print("Matriz Omega (5x5 para N=1):")
    print(omega_full)
    print("\nMatriz P (Cholesky, Omega = P * P'):")
    print(P)
    print("\nVerificacion P * P' = Omega:", np.allclose(P @ P.T, omega_full))

    # Simulacion
    beta0_gls  = np.zeros(N_SIM);  beta1_gls  = np.zeros(N_SIM)
    beta0_fgls = np.zeros(N_SIM);  beta1_fgls = np.zeros(N_SIM)

    rng = np.random.RandomState(SEED)

    for sim in range(N_SIM):
        y, x = generate_data(n_total, BETA0_TRUE, BETA1_TRUE, rng)
        X = np.column_stack([np.ones(n_total), x])

        # --- GLS via Cholesky (oracle): transformar y estimar OLS ---
        y_star = P_inv @ y
        X_star = P_inv @ X
        beta_cholesky = ols(y_star, X_star)
        beta0_gls[sim] = beta_cholesky[0]
        beta1_gls[sim] = beta_cholesky[1]

        # --- FGLS grupos (punto 1a) ---
        beta_fgls_hat, _ = fgls(y, X, n_total)
        beta0_fgls[sim] = beta_fgls_hat[0]
        beta1_fgls[sim] = beta_fgls_hat[1]

    # Reportar comparacion
    W = 65
    print("\n" + "=" * W)
    print(f"  GLS (Cholesky, oracle) vs FGLS grupos  |  5N = {n_total}")
    print("=" * W)

    print("\n--- beta0 (verdadero = -3.0) ---")
    print(f"  {'':20s} {'GLS (Cholesky)':>16s} {'FGLS grupos':>14s}")
    print(f"  {'Media':20s} {np.mean(beta0_gls):16.6f} {np.mean(beta0_fgls):14.6f}")
    print(f"  {'Mediana':20s} {np.median(beta0_gls):16.6f} {np.median(beta0_fgls):14.6f}")
    print(f"  {'Desvio':20s} {np.std(beta0_gls):16.6f} {np.std(beta0_fgls):14.6f}")

    print("\n--- beta1 (verdadero = 0.8) ---")
    print(f"  {'':20s} {'GLS (Cholesky)':>16s} {'FGLS grupos':>14s}")
    print(f"  {'Media':20s} {np.mean(beta1_gls):16.6f} {np.mean(beta1_fgls):14.6f}")
    print(f"  {'Mediana':20s} {np.median(beta1_gls):16.6f} {np.median(beta1_fgls):14.6f}")
    print(f"  {'Desvio':20s} {np.std(beta1_gls):16.6f} {np.std(beta1_fgls):14.6f}")

run_simulation_1b(n_total=5)


#### 2.1a) Diseno 0 (tamano) + 2.1b) Disenos 1 y 2 (poder) para todos los n

In [13]:
# Correr test de White para todos los tamanos de muestra y los 3 disenos
sample_sizes_p2 = [20, 60, 100, 200, 400, 600]

for n in sample_sizes_p2:
    res = run_white_test_simulation(n, designs=[0, 1, 2])
    print_white_results(res, n)
    print()

  Test de White - n = 20

  Diseno 0 (homocedasticidad) - TAMANO del test:
    Al  1%: 0.48%  (nominal:  1%)
    Al  5%: 4.60%  (nominal:  5%)
    Al 10%: 9.04%  (nominal: 10%)

  Diseno 1 (normal + heterocedasticidad) - PODER del test:
    Al  1%: 0.60%
    Al  5%: 6.00%
    Al 10%: 11.38%

  Diseno 2 (t5 + heterocedasticidad) - PODER del test:
    Al  1%: 0.76%
    Al  5%: 6.92%
    Al 10%: 12.26%

  Test de White - n = 60

  Diseno 0 (homocedasticidad) - TAMANO del test:
    Al  1%: 0.90%  (nominal:  1%)
    Al  5%: 4.86%  (nominal:  5%)
    Al 10%: 9.94%  (nominal: 10%)

  Diseno 1 (normal + heterocedasticidad) - PODER del test:
    Al  1%: 5.14%
    Al  5%: 14.66%
    Al 10%: 23.86%

  Diseno 2 (t5 + heterocedasticidad) - PODER del test:
    Al  1%: 3.44%
    Al  5%: 11.20%
    Al 10%: 18.06%

  Test de White - n = 100

  Diseno 0 (homocedasticidad) - TAMANO del test:
    Al  1%: 1.08%  (nominal:  1%)
    Al  5%: 4.70%  (nominal:  5%)
    Al 10%: 9.78%  (nominal: 10%)

  Diseno 1 

### 2.2 Correccion de la matriz de varianzas y covarianzas (White)

Sesgo relativo: $b_j = \frac{1}{S}\sum_{s=1}^{S} \frac{\widehat{Var}(\hat\beta_j)^{(s)} - Var(\hat\beta_j)}{Var(\hat\beta_j)}$

Sesgo relativo total: $|b_0| + |b_1| + |b_2|$

In [38]:
def white_variance_simulation(n, design, use_true_errors=False, rng_seed=SEED):
    """
    Parte 2.2: Sesgo relativo de la estimacion de White de la matriz de var-cov.
    
    Var_true = (X'X)^{-1} X' Omega_true X (X'X)^{-1}  (varianza verdadera de OLS)
    Var_hat  = (X'X)^{-1} X' Omega_hat  X (X'X)^{-1}  (estimacion de White)
    
    Omega_hat = diag(e_hat^2) si use_true_errors=False (residuos)
    Omega_hat = diag(epsilon^2) si use_true_errors=True (errores verdaderos)
    """
    rng_x = np.random.RandomState(rng_seed)
    x1, x2 = generate_x_part2(n, rng_x)
    X = np.column_stack([np.ones(n), x1, x2])
    
    # Varianza de u segun diseno
    if design == 1:
        var_u = 1.0  # u ~ N(0,1)
    elif design == 2:
        var_u = 5.0 / 3.0  # u ~ t5, Var(t5) = 5/(5-2)
    
    # nu_i para heterocedasticidad
    nu = np.exp(0.25 * x1 + 0.25 * x2)
    
    # Varianza verdadera de epsilon_i = sqrt(nu_i)*u_i es: nu_i * Var(u)
    sigma2_true = nu * var_u
    Omega_true = np.diag(sigma2_true)
    
    # Varianza verdadera de beta_hat OLS: (X'X)^{-1} X' Omega_true X (X'X)^{-1}
    XtX_inv = np.linalg.inv(X.T @ X)
    Var_true = XtX_inv @ X.T @ Omega_true @ X @ XtX_inv
    var_true_diag = np.diag(Var_true)  # [Var(b0), Var(b1), Var(b2)]
    
    # Simulacion
    rng = np.random.RandomState(rng_seed)
    relative_bias = np.zeros((N_SIM, 3))  # para b0, b1, b2
    
    for sim in range(N_SIM):
        # Generar errores
        if design == 1:
            u = rng.normal(0, 1, size=n)
        elif design == 2:
            u = rng.standard_t(df=5, size=n)
        
        epsilon = np.sqrt(nu) * u
        y = 1 + 1 * x1 + 1 * x2 + epsilon
        
        # OLS
        beta_hat = ols(y, X)
        e_hat = y - X @ beta_hat
        
        # Estimacion de White
        if use_true_errors:
            Omega_hat = np.diag(epsilon ** 2)  # errores verdaderos
        else:
            Omega_hat = np.diag(e_hat ** 2)    # residuos OLS
        
        Var_hat = XtX_inv @ X.T @ Omega_hat @ X @ XtX_inv
        var_hat_diag = np.diag(Var_hat)
        
        # Sesgo relativo para esta simulacion
        relative_bias[sim, :] = (var_hat_diag - var_true_diag) / var_true_diag
    
    # Promediar sobre simulaciones
    b = np.mean(relative_bias, axis=0)  # b0, b1, b2
    total_bias = np.sum(np.abs(b))
    
    return b, total_bias


def run_white_variance_all(use_true_errors=False):
    """Corre la simulacion 2.2 para todos los n y disenos."""
    sample_sizes = [20, 60, 100, 200, 400, 600]
    label = "errores verdaderos" if use_true_errors else "residuos OLS"
    
    print("=" * 70)
    print(f"  Sesgo relativo de White ({label})")
    print("=" * 70)
    
    for design in [1, 2]:
        print(f"\n--- Diseno {design} ---")
        print(f"  {'n':>5s}  {'b0':>10s}  {'b1':>10s}  {'b2':>10s}  {'|b0|+|b1|+|b2|':>16s}")
        
        for n in sample_sizes:
            b, total = white_variance_simulation(n, design, use_true_errors)
            print(f"  {n:5d}  {b[0]:10.4f}  {b[1]:10.4f}  {b[2]:10.4f}  {total:16.4f}")


# a-f) Con residuos OLS (estimacion estandar de White)
run_white_variance_all(use_true_errors=False)

  Sesgo relativo de White (residuos OLS)

--- Diseno 1 ---
      n          b0          b1          b2    |b0|+|b1|+|b2|
     20     -0.1592     -0.2099     -0.2123            0.5813
     60     -0.0511     -0.0677     -0.0644            0.1831
    100     -0.0317     -0.0430     -0.0407            0.1154
    200     -0.0160     -0.0204     -0.0204            0.0568
    400     -0.0070     -0.0093     -0.0088            0.0251
    600     -0.0048     -0.0055     -0.0067            0.0171

--- Diseno 2 ---
      n          b0          b1          b2    |b0|+|b1|+|b2|
     20     -0.1546     -0.2071     -0.2226            0.5843
     60     -0.0614     -0.0730     -0.0790            0.2133
    100     -0.0364     -0.0443     -0.0466            0.1273
    200     -0.0182     -0.0204     -0.0250            0.0636
    400     -0.0094     -0.0093     -0.0140            0.0327
    600     -0.0049     -0.0036     -0.0088            0.0173


#### 2.2g) Repetir con errores verdaderos en vez de residuos

In [39]:
# g) Con errores verdaderos (epsilon^2 en vez de e_hat^2)
run_white_variance_all(use_true_errors=True)

  Sesgo relativo de White (errores verdaderos)

--- Diseno 1 ---
      n          b0          b1          b2    |b0|+|b1|+|b2|
     20      0.0018     -0.0008      0.0144            0.0169
     60      0.0006      0.0011      0.0069            0.0086
    100     -0.0006     -0.0015      0.0031            0.0052
    200     -0.0003      0.0004      0.0014            0.0021
    400      0.0008      0.0008      0.0019            0.0034
    600      0.0004      0.0013      0.0005            0.0023

--- Diseno 2 ---
      n          b0          b1          b2    |b0|+|b1|+|b2|
     20     -0.0018     -0.0109     -0.0158            0.0286
     60     -0.0097     -0.0067     -0.0086            0.0250
    100     -0.0050     -0.0039     -0.0036            0.0125
    200     -0.0025     -0.0006     -0.0042            0.0073
    400     -0.0017      0.0005     -0.0035            0.0057
    600      0.0003      0.0030     -0.0019            0.0052
